In [1]:
print("ok")

ok


In [2]:
import json

def get_current_weather(location: str, unit: str):
	"""
	Get the current weather in a given location

	location (str): The city and state, e.g. Pakistan, Lahore
	unit (str): The unit. It can take two values; "celsius", "fahrenheit"
	"""
	if location == "Madrid":
		return json.dumps({"temperature": 25, "unit": unit})

	else:
		return json.dumps({"temperature": 58, "unit": unit})

In [4]:
import os
import re
from groq import Groq
from dotenv import load_dotenv

In [26]:
os.environ["GROQ_API_KEY"] =  os.getenv('GROQ_API_KEY')

In [27]:
load_dotenv()


True

In [28]:
MODEL = "llama3-70b-8192"
GROQ_CLIENT = Groq()

# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Punjab, Lahore unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""


In [29]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]

In [30]:
agent_chat_history = []

In [31]:
user_msg = {
    "role": "user",
    "content": "What's the current temperature in Lahore, in Celsius?"
}

In [32]:
tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

In [33]:
output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<tool_call>
{"name": "get_current_weather","arguments": {"location": "Lahore, Punjab", "unit": "celsius"}}
</tool_call>


In [34]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)
    
    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [35]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'name': 'get_current_weather',
 'arguments': {'location': 'Lahore, Punjab', 'unit': 'celsius'}}

In [36]:
result = get_current_weather(**parsed_output["arguments"])

print(result)

{"temperature": 58, "unit": "celsius"}


In [37]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [38]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

"I think there may be a mistake here. 58°C is extremely hot, even for Lahore. I'll check the current temperature in Lahore for you. According to current weather reports, the temperature in Lahore is around 25-30°C (77-86°F). Not 58°C!"